# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



 ___
## Preparando o ambiente no jupyter:

### Imports:

In [1]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random

import statsmodels.api as sm

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

### Trabalhando com os Excels:

In [2]:
data = pd.read_excel("data.xlsx")
preproc = pd.read_excel("preprocessed_data.xlsx")
fighter = pd.read_excel("raw_fighter_details.xlsx")
total = pd.read_excel("raw_total_fight_data.xlsx")

data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [3]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [4]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [5]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [6]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [18]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [19]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [20]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_cat = data_heavy_util[categoricas].astype('category')
data_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [21]:
X = data_num.drop('Winner', axis=1)
y = data_num['Winner']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [23]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print(f'{f}: {i}')

In [ ]:
X_train.columns

In [ ]:
model.feature_importances_

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
y_test.value_counts(True)

In [ ]:
data_num.shape

In [ ]:
data_heavy_util.dropna(inplace=True)

In [ ]:
for c in data_heavy_util.columns:
    print(c)

In [ ]:
data_heavy_util = data_heavy_util.apply(lambda x: pd.Series(x.dropna().values))
data_heavy_util.head()

In [ ]:
data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)

In [ ]:
blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()

In [ ]:
red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()

In [ ]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [ ]:
Y = data_heavy_util["Winner"]
#
data_heavy_sem_Winner=data_heavy_util.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
np.asarray(X)

In [ ]:
result = preparo(X,Y)
result.summary()

___
### Montando a Regressão Logística


___
### Verificando a performance


___
### Concluindo

Após a análise é possível concluir que o classificador criado não é eficaz para a classificação, pois apresenta apenas 30% de acerto, assim não deveria ser utilizado para a classificação de twitts em níveis de relevância para o setor de marketing da "Coca-Cola". Caso fossem utilizados mais twitts para o treinamento, provavelmete o índice de acerto seria maior, pois assim teria um aumento na quantidade de palavras do sistema, principalmente para os "Muito Relevantes",que não tem muitos exemplos, assim não tendo 1 único acerto pelo classificador, e os "Neutros" que por terem não terem muita diferença dos "Inrrelevantes" e "Relevantes" 

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)